In [1]:
!pip install diffusers accelerate transformers datasets -q


In [2]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from transformers import AutoTokenizer
from datasets import load_dataset


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [3]:
from diffusers import StableDiffusionPipeline

model_name = "CompVis/stable-diffusion-v1-4"  # Replace with lightweight version
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
).to("cuda")


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

tokenizer/tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
# Define your image folder path
image_folder_path = "/kaggle/input/skin-cancer-img-to-img/all_image_cancer"

# Preprocessing function
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return image


In [5]:
# Load images from the directory
from PIL import Image
import os
image_files = [os.path.join(image_folder_path, f) for f in os.listdir(image_folder_path) if f.endswith('.jpg') or f.endswith('.png')]

# Preprocess all images
images = [preprocess_image(img_path) for img_path in image_files]


In [6]:
# Select a sample image for testing (using the first image in the folder)
input_image = images[0]


In [7]:
# Define the prompt
prompt = "A high-quality transformation of a skin cancer image"

# Call the pipeline with the input image and other parameters
result = pipeline(prompt=prompt, image=input_image, strength=0.5, guidance_scale=7.5)


  0%|          | 0/25 [00:00<?, ?it/s]

In [8]:
# Print the result to understand its structure
print(result)

# Try extracting the image from the correct key
if "images" in result:
    output_image = result["images"][0]  # Extract the first image from the list
else:
    print("Error: 'images' key not found in the result.")


StableDiffusionPipelineOutput(images=[<PIL.Image.Image image mode=RGB size=600x448 at 0x7EBCCF011240>], nsfw_content_detected=[False])


In [9]:
# Check if the output image is a valid PIL Image and save it
if isinstance(output_image, Image.Image):
    output_image.save("output_image.jpg")
    print("Image saved successfully.")
else:
    print("Error: The result is not a PIL Image object or was not processed correctly.")


Image saved successfully.


In [10]:
# Save the fine-tuned model
pipeline.save_pretrained("./fine_tuned_model")
print("Model saved successfully.")


Model saved successfully.


In [11]:
from skimage.metrics import structural_similarity as ssim
import numpy as np
from PIL import Image

# Function to calculate SSIM between input and generated image
def calculate_ssim(input_img, generated_img):
    # Resize generated image to the same size as the input image
    generated_img = generated_img.resize(input_img.size)
    
    # Convert images to NumPy arrays
    input_img = np.array(input_img)
    generated_img = np.array(generated_img)
    
    # Calculate SSIM with a smaller window size to avoid errors for small images
    return ssim(input_img, generated_img, multichannel=True, win_size=3)

# Calculate SSIM between input image and generated image
ssim_value = calculate_ssim(input_image, output_image)
print(f"SSIM: {ssim_value}")


SSIM: 0.9010540592630195
